In [1]:
from datasets import load_dataset
ds = load_dataset("cfilt/iitb-english-hindi")

/home/hxman/Work/Subtitle-Generator/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# print(ds)
print(ds['train'][0])
# print(ds['test'][0])
# print(ds['validation'][0])

{'translation': {'en': 'Give your application an accessibility workout', 'hi': 'अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें'}}


In [3]:
train_pairs = []
for item in ds['train']['translation']:
    train_pairs.append({
        'target': item['hi'],
        'source': item['en']
    })
print(train_pairs[:5])

test_pairs = []
for item in ds['test']['translation']:
    test_pairs.append({
        'target': item['hi'],
        'source': item['en']
    })
print(test_pairs[:5])

validation_pairs = []
for item in ds['validation']['translation']:
    validation_pairs.append({
        'target': item['hi'],
        'source': item['en']
    })
print(validation_pairs[:5])

[{'target': 'अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें', 'source': 'Give your application an accessibility workout'}, {'target': 'एक्सेर्साइसर पहुंचनीयता अन्वेषक', 'source': 'Accerciser Accessibility Explorer'}, {'target': 'निचले पटल के लिए डिफोल्ट प्लग-इन खाका', 'source': 'The default plugin layout for the bottom panel'}, {'target': 'ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका', 'source': 'The default plugin layout for the top panel'}, {'target': 'उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से निष्क्रिय किया गया है', 'source': 'A list of plugins that are disabled by default'}]
[{'target': 'आपकी कार में ब्लैक बॉक्स?', 'source': 'A black box in your car?'}, {'target': 'जबकि अमेरिका के सड़क योजनाकार, ध्वस्त होते हुए हाईवे सिस्टम को सुधारने के लिए धन की कमी से जूझ रहे हैं, वहीं बहुत-से लोग इसका समाधान छोटे से ब्लैक बॉक्स में देख रहे हैं, जो आपकी कार के डैशबोर्ड पर सफ़ाई से फिट हो जाता है।', 'source': "As America's road planners struggle to find the cash to mend a crumbling highway system, man

In [4]:
import re
def clean(text):
    text = text.strip()
    text = re.sub(r'\s+', ' ', text)
    return text
def noise_filter(pairs):
  pairs = [p for p in pairs 
           if len(p["source"].split()) <= 80 
           and len(p["target"].split()) <= 80]
  return pairs

train_pairs = noise_filter(train_pairs)
validation_pairs = noise_filter(validation_pairs)
test_pairs = noise_filter(test_pairs)

In [5]:
import json
f = open("../data/raw/train-en-hi.json", 'w+')
json.dump(train_pairs, f, ensure_ascii=False, indent=4)
f.close()
f = open("../data/raw/validation-en-hi.json", "w+")
json.dump(validation_pairs, f, ensure_ascii=False, indent=4)
f.close()
f = open("../data/raw/test-en-hi.json", "w+")
json.dump(test_pairs, f, ensure_ascii=False, indent=4)
f.close()

In [6]:
with open("../data/raw/spm_corpus.txt", "w", encoding="utf-8") as f:
    for p in train_pairs:
        f.write(p["source"] + "\n")
        f.write(p["target"] + "\n")

In [1]:
# RUN THIS IN COLAB
# import os
# import sentencepiece as spm

# os.makedirs("../data/tokenizer", exist_ok=True)

# spm.SentencePieceTrainer.Train(
#     input="../data/raw/spm_corpus.txt",
#     model_prefix="../data/tokenizer/en_hi_spm",
#     vocab_size=32000,
#     character_coverage=1.0,
#     model_type='bpe'
# )

# print("Tokenizer trained & saved.")

In [4]:
import sentencepiece as spm

sp = spm.SentencePieceProcessor()
sp.load("../models/translation/en_hi_spm.model")

True

In [6]:
import json

with open("../data/raw/train-en-hi.json", encoding="utf-8") as f:
    train_pairs = json.load(f)

tokenized_train = []
for p in train_pairs:
    src_ids = sp.encode(p["source"], out_type=int)
    tgt_ids = sp.encode(p["target"], out_type=int)
    tokenized_train.append({"src_ids": src_ids, "tgt_ids": tgt_ids})

with open("../data/raw/test-en-hi.json", encoding="utf-8") as f:
    test_pairs = json.load(f)
tokenized_test = []
for p in test_pairs:
    src_ids = sp.encode(p["source"], out_type=int)
    tgt_ids = sp.encode(p["target"], out_type=int)
    tokenized_test.append({"src_ids": src_ids, "tgt_ids": tgt_ids})

with open("../data/raw/validation-en-hi.json", encoding="utf-8") as f:
    validation_pairs = json.load(f)
tokenized_validation = []
for p in validation_pairs:
    src_ids = sp.encode(p["source"], out_type=int)
    tgt_ids = sp.encode(p["target"], out_type=int)
    tokenized_validation.append({"src_ids": src_ids, "tgt_ids": tgt_ids})
